In [9]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
!pip install wandb -qqq
import wandb
wandb.login()
api = wandb.Api()
import os

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"


selected_tag = 'Pred'

runs = api.runs("hokarami/TEDAM3")


True

In [11]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list = [], [], []
    for run in runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list
        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [12]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag=None)

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

,summary,config,name
0,{},"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R6-mc1]TEDA__shpmarklabel-1227776
1,{},"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R6-mc1]TEDA__shpmarklabel-1221958
2,{},"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R6-mc1]TEDA__shpmarklabel-1225429
3,{},"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R6-mc1]TEDA__shpmarklabel-1228455
4,{},"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[R6-mc1]TEDA__shpmarklabel-1225840
...,...,...,...
654,"{'_runtime': 923.3067276477814, 'max_obj_val':...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[sseft1]DA__label-1085186-none-state1-per100
655,"{'max_obj_val': 0.407445708376422, 'Valid-Next...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[wd4]DA__label-1084470-none-state1-per20
656,"{'_wandb': {'runtime': 175}, '_runtime': 176.4...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[wd2]DA__label-1084470-none-state1-per20
657,{'Valid-NextTime/rmse_norm': 0.330295279192351...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[wd3]DA__label-1084470-none-state1-per20


dict_keys([])

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

# Pred

In [13]:
df_filt = dl_runs(runs, selected_tag='Pred6')

In [14]:
i_groupby_row =['dataset','setting','test_center']
i_groupby_col = ['event_enc']
i_agg = ['split']
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")

,dataset,setting,test_center,event_enc,split,Best-Test-pred_label/f1-binary,Best-Test-pred_label/AUPRC,Best-Test-pred_label/AUROC
0,/scratch/hokarami/data_tedam/p19/,mc1,1,1,2,NaN,NaN,NaN
1,/scratch/hokarami/data_tedam/p19/,mc1,1,1,3,NaN,NaN,NaN
2,/scratch/hokarami/data_tedam/p19/,mc1,1,1,1,NaN,NaN,NaN
3,/scratch/hokarami/data_tedam/p19/,mc1,1,1,0,NaN,NaN,NaN
4,/scratch/hokarami/data_tedam/p19/,mc1,1,1,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
59,/scratch/hokarami/data_tedam/p12/,mc1,0,1,2,0.438503,0.485630,0.876547
60,/scratch/hokarami/data_tedam/p12/,mc1,0,1,4,0.525000,0.536934,0.890376
61,/scratch/hokarami/data_tedam/p12/,mc1,0,1,3,0.496124,0.566278,0.861949
62,/scratch/hokarami/data_tedam/p19/,sc,0,0,0,0.690110,0.793120,0.942848


Best-Test-pred_label/f1-binary  \
dataset                           setting test_center event_enc                                  
/scratch/hokarami/data_tedam/p12/ mc1     0           0                            0.49 (0.03)   
                                                      1                            0.49 (0.03)   
                                  sc      1           0                             0.44 (nan)   
                                  tl      0           1                            0.43 (0.05)   
/scratch/hokarami/data_tedam/p19/ mc1     0           0                            0.68 (0.04)   
                                                      1                            0.76 (0.06)   
                                          1           0                            0.81 (0.03)   
                                                      1                            0.82 (0.06)   
                                  sc      0           0                             0.69 (nan)   

                                                                Best-Test-pred_label/AUPRC  \
dataset                           setting test_center event_enc                              
/scratch/hokarami/data_tedam/p12/ mc1     0           0                        0.49 (0.06)   
                                                      1                        0.51 (0.05)   
                                  sc      1           0                         0.46 (nan)   
                                  tl      0           1                         0.4 (0.07)   
/scratch/hokarami/data_tedam/p19/ mc1     0           0                         0.9 (0.02)   
                                                      1                        0.92 (0.04)   
                                          1           0                        0.91 (0.02)   
                                                      1                        0.93 (0.06)   
                                  sc      0           0                         0.79 (nan)   

                                                                Best-Test-pred_label/AUROC  
dataset                           setting test_center event_enc                             
/scratch/hokarami/data_tedam/p12/ mc1     0           0                        0.86 (0.02)  
                                                      1                        0.87 (0.02)  
                                  sc      1           0                         0.82 (nan)  
                                  tl      0           1                         0.8 (0.04)  
/scratch/hokarami/data_tedam/p19/ mc1     0           0                         0.98 (0.0)  
                                                      1                        0.99 (0.01)  
                                          1           0                         0.99 (0.0)  
                                                      1                        0.99 (0.01)  
                                  sc      0           0                         0.94 (nan)

Best-Test-pred_label/f1-binary  \
event_enc                                                                          0   
dataset                           setting test_center                                  
/scratch/hokarami/data_tedam/p12/ mc1     0                              0.49 (0.03)   
                                  sc      1                               0.44 (nan)   
                                  tl      0                                      NaN   
/scratch/hokarami/data_tedam/p19/ mc1     0                              0.68 (0.04)   
                                          1                              0.81 (0.03)   
                                  sc      0                               0.69 (nan)   

                                                                    \
event_enc                                                        1   
dataset                           setting test_center                
/scratch/hokarami/data_tedam/p12/ mc1     0            0.49 (0.03)   
                                  sc      1                    NaN   
                                  tl      0            0.43 (0.05)   
/scratch/hokarami/data_tedam/p19/ mc1     0            0.76 (0.06)   
                                          1            0.82 (0.06)   
                                  sc      0                    NaN   

                                                      Best-Test-pred_label/AUPRC  \
event_enc                                                                      0   
dataset                           setting test_center                              
/scratch/hokarami/data_tedam/p12/ mc1     0                          0.49 (0.06)   
                                  sc      1                           0.46 (nan)   
                                  tl      0                                  NaN   
/scratch/hokarami/data_tedam/p19/ mc1     0                           0.9 (0.02)   
                                          1                          0.91 (0.02)   
                                  sc      0                           0.79 (nan)   

                                                                    \
event_enc                                                        1   
dataset                           setting test_center                
/scratch/hokarami/data_tedam/p12/ mc1     0            0.51 (0.05)   
                                  sc      1                    NaN   
                                  tl      0             0.4 (0.07)   
/scratch/hokarami/data_tedam/p19/ mc1     0            0.92 (0.04)   
                                          1            0.93 (0.06)   
                                  sc      0                    NaN   

                                                      Best-Test-pred_label/AUROC  \
event_enc                                                                      0   
dataset                           setting test_center                              
/scratch/hokarami/data_tedam/p12/ mc1     0                          0.86 (0.02)   
                                  sc      1                           0.82 (nan)   
                                  tl      0                                  NaN   
/scratch/hokarami/data_tedam/p19/ mc1     0                           0.98 (0.0)   
                                          1                           0.99 (0.0)   
                                  sc      0                           0.94 (nan)   

                                                                    
event_enc                                                        1  
dataset                           setting test_center               
/scratch/hokarami/data_tedam/p12/ mc1     0            0.87 (0.02)  
                                  sc      1                    NaN  
                                  tl      0             0.8 (0.04)  
/scratch/hokarami/data_tedam/p19/ mc1     0            0.99 (0.01)  
  

# CIF

In [7]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

dict_keys(['w', 'lr', 'log', 'mod', 'per', 'cuda', 'data', 'date', 'demo', 'prof', 'epoch', 'split', 'state', 'w_pos', 'wandb', 'ES_pat', 'device', 'freeze', 'run_id', 'smooth', 'te_d_k', 'te_d_v', 'w_time', 'dataset', 'int_dec', 'setting', 'w_class', 'w_event', 'run_name', 'run_path', 'te_d_rnn', 'time_enc', 'TE_config', 'event_enc', 'next_mark', 'num_demos', 'num_marks', 'num_types', 'pos_alpha', 'te_d_mark', 'te_d_time', 'te_n_head', 'type_loss', 'CIF_config', 'DAM_config', 'batch_size', 'data_label', 'num_states', 'pos_weight', 'str_config', 'te_d_inner', 'te_dropout', 'testloader', 'demo_config', 'mark_detach', 'te_n_layers', 'test_center', 'trainloader', 'user_prefix', 'validloader', 'w_pos_label', 'label_config', 'sample_label', 'weight_decay', 'hparams2write', 'balanced_batch', 'label_loss_fun', 'pred_loss_func', 'w_sample_label', 'next_time_config', 'next_type_config', 'transfer_learning'])

In [9]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")

,dataset,setting,state,test_center,split,Test-CIF/NLL-#events
0,/scratch/hokarami/data_tedam/p19/,mc1,True,1,2,0.607923
1,/scratch/hokarami/data_tedam/p19/,mc1,True,1,0,0.698569
2,/scratch/hokarami/data_tedam/p19/,mc1,True,1,1,0.785800
3,/scratch/hokarami/data_tedam/p19/,mc1,True,1,4,0.705369
4,/scratch/hokarami/data_tedam/p19/,mc1,True,1,3,0.749281
...,...,...,...,...,...,...
135,/scratch/hokarami/data_tedam/p19/,sc,False,0,2,0.667987
136,/scratch/hokarami/data_tedam/p19/,sc,False,0,1,0.641528
137,/scratch/hokarami/data_tedam/p19/,sc,False,0,0,0.691973
138,/scratch/hokarami/data_tedam/p19/,sc,False,0,3,0.750535


Test-CIF/NLL-#events
dataset                           setting state                     
/scratch/hokarami/data_tedam/p12/ mc1     False         -0.4 (0.138)
                                          True         -0.797 (0.13)
                                  mc2     False       -0.312 (0.083)
                                          True        -0.781 (0.046)
                                  sc      False        0.075 (0.073)
                                          True        -0.329 (0.111)
/scratch/hokarami/data_tedam/p19/ mc1     False          0.808 (0.2)
                                          True         0.582 (0.146)
                                  mc2     False        0.988 (0.148)
                                          True         0.857 (0.164)
                                  sc      False         0.94 (0.275)
                                          True          0.68 (0.299)

Test-CIF/NLL-#events                
state                                                    False            True
dataset                           setting                                     
/scratch/hokarami/data_tedam/p12/ mc1             -0.4 (0.138)   -0.797 (0.13)
                                  mc2           -0.312 (0.083)  -0.781 (0.046)
                                  sc             0.075 (0.073)  -0.329 (0.111)
/scratch/hokarami/data_tedam/p19/ mc1              0.808 (0.2)   0.582 (0.146)
                                  mc2            0.988 (0.148)   0.857 (0.164)
                                  sc              0.94 (0.275)    0.68 (0.299)